In [20]:
import pandas as pd
import numpy as np

In [2]:
event_df = pd.read_csv("../dataset/events_processed.csv")
item_properties_part_1_df = pd.read_csv("../dataset/item_properties_part1_processed.csv")
item_properties_part_2_df = pd.read_csv("../dataset/item_properties_part2_processed.csv")
event_df.head()
event_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
 5   ISO datetime   object 
 6   date           object 
dtypes: float64(1), int64(3), object(3)
memory usage: 147.2+ MB


In [3]:
item_properties_df = pd.concat([item_properties_part_1_df, item_properties_part_2_df], ignore_index=True)
item_properties_df.head()

,timestamp,itemid,property,value,ISO datetime,date
0,1435460400000,460429,categoryid,1338,2015-06-28 03:00:00,2015-06-28
1,1441508400000,206783,888,1116713 960601 n277.200,2015-09-06 03:00:00,2015-09-06
2,1439089200000,395014,400,n552.000 639502 n720.000 424566,2015-08-09 03:00:00,2015-08-09
3,1431226800000,59481,790,n15360.000,2015-05-10 03:00:00,2015-05-10
4,1431831600000,156781,917,828513,2015-05-17 03:00:00,2015-05-17


### RFM:  A behavioral segmentation technique used to quantify customer value by analyzing past purchase behavior

* R- Recency

In [4]:
transactions_df = event_df[event_df['event'] == 'transaction'].copy()
transactions_df["ISO datetime"] = pd.to_datetime(transactions_df["ISO datetime"])
transactions_df.head()
# transactions_df.info()
# print(transactions_df["ISO datetime"].dtype)

,timestamp,visitorid,event,itemid,transactionid,ISO datetime,date
130,1433222276276,599528,transaction,356475,4000.0,2015-06-02 05:17:56.276,2015-06-02
304,1433193500981,121688,transaction,15335,11117.0,2015-06-01 21:18:20.981,2015-06-01
418,1433193915008,552148,transaction,81345,5444.0,2015-06-01 21:25:15.008,2015-06-01
814,1433176736375,102019,transaction,150318,13556.0,2015-06-01 16:38:56.375,2015-06-01
843,1433174518180,189384,transaction,310791,7244.0,2015-06-01 16:01:58.180,2015-06-01


In [5]:
# Xác định Ngày Tham Chiếu (NOW)
latest_purchase_date = transactions_df['ISO datetime'].max()
print(latest_purchase_date)
NOW = latest_purchase_date + pd.Timedelta(days=1) # Tăng thêm 1 ngày để đảm bảo tất cả các giao dịch đều trước Ngày Tham Chiếu

2015-09-18 02:43:12.017000


In [11]:
rfm_r_df = transactions_df.groupby('visitorid')['ISO datetime'].max().reset_index()    # ngày mua gần nhất
rfm_r_df.columns = ['visitorid', 'LastPurchaseDate']  
rfm_r_df['Recency'] = (NOW - rfm_r_df['LastPurchaseDate']).dt.days
rfm_r_df = rfm_r_df[['visitorid', 'Recency']]
rfm_r_df.head()

,visitorid,Recency
0,172,35
1,186,37
2,264,11
3,419,51
4,539,94


In [ ]:
# visualization of Recency

* F - Frequency 

In [15]:
# Tính toán Frequency (F) 
# Đếm số lượng transactionid DUY NHẤT cho mỗi visitorid
rfm_f_df = transactions_df.groupby('visitorid')['transactionid'].nunique().reset_index()
rfm_f_df.columns = ['visitorid', 'Frequency']
print(rfm_f_df.head())
rfm_f_df.describe()

   visitorid  Frequency
0        172          1
1        186          1
2        264          1
3        419          1
4        539          1


,visitorid,Frequency
count,1.171900e+04,11719.000000
mean,7.099856e+05,1.507978
std,4.058287e+05,7.344491
min,1.720000e+02,1.000000
25%,3.591195e+05,1.000000
50%,7.183140e+05,1.000000
75%,1.062739e+06,1.000000
max,1.407398e+06,502.000000


In [ ]:
# visualization of Frequency

* M - Monetary

In [36]:
# Tính toán Monetary (M)
price_df = item_properties_df[item_properties_df['property'] == '790'].copy()
price_df['ISO datetime'] = pd.to_datetime(price_df['ISO datetime'])
price_df.head()

,timestamp,itemid,property,value,ISO datetime,date
3,1431226800000,59481,790,n15360.000,2015-05-10 03:00:00,2015-05-10
14,1434250800000,169055,790,n21000.000,2015-06-14 03:00:00,2015-06-14
16,1435460400000,178601,790,n5400.000,2015-06-28 03:00:00,2015-06-28
42,1431831600000,125874,790,n39588.000,2015-05-17 03:00:00,2015-05-17
46,1433646000000,272201,790,n10320.000,2015-06-07 03:00:00,2015-06-07


In [37]:
# Làm sạch cột 'value' (loại bỏ 'n' và chuyển sang float)
price_data_clean = price_df.copy()
price_data_clean['clean_value'] = price_data_clean['value'].apply(
    lambda x: x.replace('n', '', 1) if isinstance(x, str) and x.startswith('n') else np.nan
)
price_data_clean['price'] = pd.to_numeric(price_data_clean['clean_value'], errors='coerce')
print(price_data_clean.head(2))
price_df = price_data_clean[['ISO datetime', 'itemid', 'price']].rename(columns={'ISO datetime': 'PriceISO'})
price_df

        timestamp  itemid property       value        ISO datetime  \
3   1431226800000   59481      790  n15360.000 2015-05-10 03:00:00   
14  1434250800000  169055      790  n21000.000 2015-06-14 03:00:00   

          date clean_value    price  
3   2015-05-10   15360.000  15360.0  
14  2015-06-14   21000.000  21000.0  


,PriceISO,itemid,price
3,2015-05-10 03:00:00,59481,15360.0
14,2015-06-14 03:00:00,169055,21000.0
16,2015-06-28 03:00:00,178601,5400.0
42,2015-05-17 03:00:00,125874,39588.0
46,2015-06-07 03:00:00,272201,10320.0
...,...,...,...
20275852,2015-06-07 03:00:00,436865,0.0
20275860,2015-07-19 03:00:00,241233,12048.0
20275867,2015-05-31 03:00:00,463381,0.0
20275876,2015-07-12 03:00:00,147935,42720.0


In [35]:
# Time-Based Merge : Ghép giá theo thời gian gần nhất 
transactions_df = transactions_df.sort_values('ISO datetime')
price_df = price_df.sort_values('PriceISO')

merged_df = pd.merge_asof(
    transactions_df,
    price_df,
    left_on='ISO datetime',
    right_on='PriceISO',
    by='itemid',
    direction='backward'
)
merged_df['price'] = merged_df['price'].fillna(0) # Gán giá 0 nếu không tìm thấy
merged_df

,timestamp,visitorid,event,itemid,transactionid,ISO datetime,date,PriceISO,price
0,1430623641391,869008,transaction,40685,9765.0,2015-05-03 03:27:21.391,2015-05-03,NaT,0.0
1,1430624101772,345781,transaction,438400,1016.0,2015-05-03 03:35:01.772,2015-05-03,NaT,0.0
2,1430625707591,586756,transaction,440917,10942.0,2015-05-03 04:01:47.591,2015-05-03,NaT,0.0
3,1430626058961,435495,transaction,175893,6173.0,2015-05-03 04:07:38.961,2015-05-03,NaT,0.0
4,1430627474903,266417,transaction,445106,12546.0,2015-05-03 04:31:14.903,2015-05-03,NaT,0.0
...,...,...,...,...,...,...,...,...,...
22452,1442542096512,152963,transaction,72462,5772.0,2015-09-18 02:08:16.512,2015-09-18,2015-09-13 03:00:00,15840.0
22453,1442542096512,152963,transaction,12504,5772.0,2015-09-18 02:08:16.512,2015-09-18,2015-09-13 03:00:00,16080.0
22454,1442542096528,152963,transaction,380196,5772.0,2015-09-18 02:08:16.528,2015-09-18,2015-09-13 03:00:00,194880.0
22455,1442543898098,152963,transaction,362697,5670.0,2015-09-18 02:38:18.098,2015-09-18,2015-05-10 03:00:00,34800.0


In [39]:
# Tính tổng Monetary
rfm_m_df = merged_df.groupby('visitorid')['price'].sum().reset_index(name='Monetary')
rfm_m_df

,visitorid,Monetary
0,172,96600.0
1,186,59880.0
2,264,94056.0
3,419,17160.0
4,539,34920.0
...,...,...
11714,1406787,54000.0
11715,1406981,143880.0
11716,1407070,71880.0
11717,1407110,20400.0


In [40]:
## Combine R, F, M into a single DataFrame
rfm_df = rfm_r_df.merge(rfm_f_df, on='visitorid').merge(rfm_m_df, on='visitorid')
rfm_df

,visitorid,Recency,Frequency,Monetary
0,172,35,1,96600.0
1,186,37,1,59880.0
2,264,11,1,94056.0
3,419,51,1,17160.0
4,539,94,1,34920.0
...,...,...,...,...
11714,1406787,109,1,54000.0
11715,1406981,123,1,143880.0
11716,1407070,128,1,71880.0
11717,1407110,45,1,20400.0


In [41]:
# Loại bỏ khách hàng có Monetary = 0
rfm_df = rfm_df[rfm_df['Monetary'] > 0].reset_index(drop=True)

# # Lưu kết quả
# rfm_df.to_csv('rfm_scores_final.csv', index=False)

print("\n--- RFM Hoàn Chỉnh Đã Sẵn Sàng ---")
print(f"Tổng số khách hàng phân khúc: {len(rfm_df)}")
# print(rfm_df.head())


--- RFM Hoàn Chỉnh Đã Sẵn Sàng ---
Tổng số khách hàng phân khúc: 10813


In [42]:
txn_items = set(transactions_df['itemid'])
prop_items = set(item_properties_df['itemid'])

missing_items = txn_items - prop_items
print(len(missing_items))
print(list(missing_items)[:10])   # xem 10 item đầu bị thiếu


380
[178176, 352258, 229382, 450587, 264221, 67614, 210975, 333869, 167985, 110645]
